In [1]:
from autokeras.image.image_supervised import load_image_dataset

#x_train, y_train = load_image_dataset(csv_file_path="/home/ubuntu/data/train.csv", images_path="/home/ubuntu/data/dataset_train_rgb")
#print(x_train.shape)
#print(y_train.shape)

x_test, y_test = load_image_dataset(csv_file_path="/home/ubuntu/data/test.csv", images_path="/home/ubuntu/data/dataset_test_rgb")
print(x_test.shape)
print(y_test.shape)

(8334, 720, 1280, 3)
(8334,)


In [2]:
x_train, y_train = load_image_dataset(csv_file_path="/home/ubuntu/data/train.csv", images_path="/home/ubuntu/data/dataset_train_rgb")
print(x_train.shape)
print(y_train.shape)

(5093, 720, 1280, 3)
(5093,)


In [11]:
from autokeras import ImageClassifier
clf = ImageClassifier(verbose=True)
clf.fit(x_train, y_train, time_limit=12 * 60 * 60)
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)


Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+


KeyboardInterrupt: 

In [90]:
import yaml
import os
import numpy as np
import cv2

def get_all_labels(input_yaml, riib=False):
    """ Gets all labels within label file

    Note that RGB images are 1280x720 and RIIB images are 1280x736.
    :param input_yaml: Path to yaml file
    :param riib: If True, change path to labeled pictures
    :return: images: Labels for traffic lights
                        List of al images with diction

    Example extract of the label file:
    ''' 
    - boxes:
        - {label: Red, occluded: false, x_max: 60.25, x_min: 45.375, y_max: 260.875, y_min: 221.0}
        - {label: Red, occluded: false, x_max: 982.625, x_min: 967.0, y_max: 240.75, y_min: 201.625}
        path: ./rgb/test/37318.png
    '''

    """
    # create data by reading the label file
    # images will be a list with all images in it.
    # Every images entry has a ['path'] property. Here, the image on the disk is saved
    # Every images entry has a ['boxes'] property. Here all labeled trafficlights for one image is saved
    # ['boxes'] has a 'label','occluded','x_max','x_min' and 'y_min' entry

    images = yaml.load(open(input_yaml, 'rb').read())

    # go through all labeled data:
    for i in range(len(images)):
        if "train" in images[i]['path']:
            images[i]['path'] = images[i]['path'].replace("./", "./dataset_train_rgb/")
        else:
            images[i]['path'] = images[i]['path'].replace("./", "./dataset_test_rgb/")
        
        # change the relativ path to an absolute path for each labeled image
        images[i]['path'] = os.path.abspath(os.path.join(os.path.dirname(input_yaml), images[i]['path']))
        
        # if riib and not rgb images are used, replace image paths and names
        if riib:
            images[i]['path'] = images[i]['path'].replace('.png', '.pgm')
            images[i]['path'] = images[i]['path'].replace('rgb/train', 'riib/train')
            images[i]['path'] = images[i]['path'].replace('rgb/test', 'riib/test')
            for box in images[i]['boxes']:
                box['y_max'] = box['y_max'] + 8
                box['y_min'] = box['y_min'] + 8

    # return list of all images inclusive labels
    return images

def extract_traffic_lights(input_yaml, crop_size = [140, 50], thresh_size = [12,4], 
            save_img =False, output_folder = "extractedTL"):
    """Function to extract all trafficlights out from an image based on the labelfile
	Trafficlights will be cut out and resized to a specified size. 
	After that, all Trafficlight images and labels will be stored and returned in a numpy array for further processing
    Trafficlights in iamgeTL and labelsTL are connected via the index of these arrays.

        :param input_yaml: [string] path and name of labelfile of TrafficLights
        :param save_img: [boolean] Specify, if extracted TL-Images should be saved in output_folder
        :param output_folder: Specify, where extracted TL-Images should be saved
        :param crop_size: Size of extracted images [height,width]. Trifficlights will be croped and resized to this size.
        :param thresh_size: Minimal values for TL to be processed [height,width]. TrafficLight with lower size will not be extracted and taken from image.
        
        :return: iamgeTL: Array of extracted TrafficLights [sample,height,width,chanel]
        :return: labelsTL: One-Hot Encoded Labels of extracted TrafficLights as follows:
                                [1,0,0,0]: Red (incl. Direction-arrows)
                                [0,1,0,0]: Yellow (incl. Direction-arrows)
                                [0,0,1,0]: Green (incl. Direction-arrows)
                                [0,0,0,1]: No TL (incl. TL which are off)
    """

    
    [thresh_y,thresh_x] = thresh_size
    [crop_height, crop_width] = crop_size
    # decode yaml-file and store information in list. 
    # For output specification of get_all_labels() see description in funtion.
    input_images = get_all_labels(input_yaml)
    input_images_new =[]
    # only take TrafficLight, with correct labels 
    for i, image_dict in enumerate(input_images):
	    # check for labels which are outside of the image and disregard them
        image_dict['boxes'][:] = [box for box in image_dict['boxes'] if not (box['x_max']>1280 or box['x_min']<0 or box['y_max']>720 or box['y_min']<0)]
        # Only regard images without with at least one labeled Traffic Light
        if image_dict['boxes'] != []:
            input_images_new.append(image_dict)
            
    input_images = input_images_new
    del input_images_new

    # extract TrafficLight from Images
    # imagesTL: ndArray for all TrafficLights: fist dimensions (NumberOfTL,CropHeight,CropWidth,Channel)
    # labelsTL: ndArray for all Labels: Label scheme specified in function description above.
    imagesTL = np.empty((0,crop_height,crop_width,3), dtype=np.uint8)
    labelsTL = []
    i=0
    # create output folder, if you want to save all images
    if not os.path.exists(output_folder) and save_img:
        os.makedirs(output_folder)
    # iterate through all images and all known Trafficlights
    for image_dict in input_images:
        for box in image_dict['boxes']:
            #boundary boxes of the TrafficLight
            boxes = [ir(box['y_min']),
                        ir(box['x_min']),
                        ir(box['y_max']),
                        ir(box['x_max'])]
            # Only take TrafficLights which are large enough and not occluded
            if boxes[2]-boxes[0] >= thresh_x and  boxes[3]-boxes[1]>= thresh_y and box['occluded']==False:
                print("prepared images: " +str(i))
                # load image
                image = cv2.imread(image_dict['path'])
                # cut TrafficLight out of image based on boundry box 
                print(image_dict['path'])
                crop_img = image[boxes[0]:boxes[2], boxes[1]:boxes[3], :]
                # resize image to crop_size
                res_img = cv2.resize(crop_img,(crop_width,crop_height))
                # append particular trafficlight-image to array with all other Trafficlights
                imagesTL = np.append(imagesTL,[res_img], axis = 0)
                # insert correct labels for particular Trafficlight
                if "Red" in box['label']:
                    labelsTL.append(1)
                elif "Yellow" in box['label']:
                    labelsTL.append(2)
                elif "Green" in box['label']:
                    labelsTL.append(3)
                else:
                    labelsTL.append(0)
                
                # you specified, safe images on disk
                if save_img == True:
                    cv2.imwrite(os.path.join(output_folder, str(i).zfill(6)+".png"), res_img)
                i=i+1

    print("Extracted TLs: " + str(np.shape(imagesTL)[0]))
    # return all preprocessed TrafficLight and corresponding Labels
    return [imagesTL, np.array(labelsTL)]

def ir(some_value):
    """litte helpfunction for integer rounding which is later needed for drawing a box:"""
    return int(round(some_value))

In [ ]:
clf.load_searcher().load_best_model().produce_keras_model().save('my_model.h5')

In [91]:
extracted_traffic_lights_train = extract_traffic_lights(input_yaml="/home/ubuntu/data/train.yaml")

prepared images: 0
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/207990.png
prepared images: 1
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/207990.png
prepared images: 2
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/207990.png
prepared images: 3
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/207990.png
prepared images: 4
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/208038.png
prepared images: 5
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/208038.png
prepared images: 6
/home/ubuntu/data/dataset_train_rgb/rgb/train/2017-02-03-11-44-56_los_altos_mountain_view_traffic_lights_bag/208038.png
prepared images: 7
/home/ub

In [44]:
print(extracted_traffic_lights_train[1])

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]]


In [92]:
extracted_traffic_lights_test = extract_traffic_lights(input_yaml="/home/ubuntu/data/test.yaml")

prepared images: 0
/home/ubuntu/data/dataset_test_rgb/rgb/test/24916.png
prepared images: 1
/home/ubuntu/data/dataset_test_rgb/rgb/test/24920.png
prepared images: 2
/home/ubuntu/data/dataset_test_rgb/rgb/test/24922.png
prepared images: 3
/home/ubuntu/data/dataset_test_rgb/rgb/test/24926.png
prepared images: 4
/home/ubuntu/data/dataset_test_rgb/rgb/test/24928.png
prepared images: 5
/home/ubuntu/data/dataset_test_rgb/rgb/test/24930.png
prepared images: 6
/home/ubuntu/data/dataset_test_rgb/rgb/test/24932.png
prepared images: 7
/home/ubuntu/data/dataset_test_rgb/rgb/test/24934.png
prepared images: 8
/home/ubuntu/data/dataset_test_rgb/rgb/test/24938.png
prepared images: 9
/home/ubuntu/data/dataset_test_rgb/rgb/test/24940.png
prepared images: 10
/home/ubuntu/data/dataset_test_rgb/rgb/test/24942.png
prepared images: 11
/home/ubuntu/data/dataset_test_rgb/rgb/test/24944.png
prepared images: 12
/home/ubuntu/data/dataset_test_rgb/rgb/test/24946.png
prepared images: 13
/home/ubuntu/data/dataset_te

In [ ]:
from autokeras import ImageClassifier

x_train = extracted_traffic_lights_train[0]
y_train = extracted_traffic_lights_train[1]
x_test = extracted_traffic_lights_test[0]
y_test = extracted_traffic_lights_test[1]

clf = ImageClassifier(verbose=True)
clf.fit(x_train, y_train, time_limit=12 * 60 * 60)
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)

Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   3.0066183894872665   |   0.8961672473867596   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+


In [89]:
print(x_train[0])
print(y_train)

[[[ 69  87  98]
  [ 68  87  97]
  [ 64  89  91]
  ...
  [ 91 186 163]
  [101 210 180]
  [103 215 183]]

 [[ 69  87  98]
  [ 68  87  97]
  [ 64  89  91]
  ...
  [ 91 186 163]
  [101 210 180]
  [103 215 183]]

 [[ 66  82  87]
  [ 66  82  86]
  [ 64  83  82]
  ...
  [ 98 176 161]
  [107 203 178]
  [108 208 181]]

 ...

 [[ 37  44  40]
  [ 37  44  40]
  [ 37  44  40]
  ...
  [ 30  34  33]
  [ 30  34  33]
  [ 30  34  33]]

 [[ 37  44  40]
  [ 37  44  40]
  [ 37  44  40]
  ...
  [ 30  34  33]
  [ 30  34  33]
  [ 30  34  33]]

 [[ 37  44  40]
  [ 37  44  40]
  [ 37  44  40]
  ...
  [ 30  34  33]
  [ 30  34  33]
  [ 30  34  33]]]
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]]
